Auteur: Michaël Leroy

[+] API: 
    
    - Nominatim: https://nominatim.openstreetmap.org/
        * obtention des surfaces en shapely geometries
    - OpenStreetMaps :
        * Lib OSMnx, obtention des résaux routiers sous forme de graph 
    - OpenChargeMap : 
        * Points de charge EV (data.gouv et autres)        

[+] Méthodologie

    - Récupération des réseaux routiers:
        * Nodes: position des intersections routiéres
        * Edges: shapely LineString, distance, limitation de vitesse, types de voies...
    - Récupération des bornes EV
        * Création d'un noeud sur l'edge le plus proche
        * Split Linestring
        # Branchement du noeud et raccord aus deux noeud 'parents'

[+] Utilité

    - Graph Stats:
        betweeness, closeness.....

[+] A ajouter pour stats:

    - On peut aussi avoir des noeuds 'amenity', ie hotels, parkings, parcs attraction...               

[-] Au dela de 3 Régions explosion de la mémoire lors de la construction du graph.

In [ ]:
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import osmnx as ox
import networkx as nx
from tqdm.notebook import tqdm

ox.settings.log_console=True
ox.settings.use_cache=True
''' To retrieve http data from OSMnx and cache them locally
    before building graph. Raise and exception if requests
    are all done. intercept then set to False and retry.'''
# ox.settings.cache_only_mode=True 

In [ ]:
# G = ox.graph_from_place('Larzicourt, France',
#     network_type='drive',
#     simplify=True,
#     retain_all=True,
#     custom_filter='["highway"~"^(trunk|primary|secondary|tertiary|motorway|motorway_link)$"]',
#     # custom_filter='["name"~"^D[0-9]+$"]',
#     # custom_filter = '{"highway": ["motorway","motorway_link"]}',
#     )
# G.number_of_nodes(), G.number_of_edges()   
# ox.plot_graph(G)
# G.nodes()
# G.remove_edge(1348792066,2263323435,0)
# ox.plot_graph(G)
# G.remove_edge(2263323435,1348792066,0)
# ox.plot_graph(G)

In [ ]:
Administratives = {
    'Auvergne-Rhône-Alpes': ['Ain', 'Allier', 'Ardèche', 'Cantal', 'Drôme', 'Isère', 'Loire', 'Haute-Loire', 'Puy-de-Dôme', 'Rhône', 'Savoie', 'Haute-Savoie'],
    'Bourgogne-Franche-Comté': ['Côte-d\'Or', 'Doubs', 'Jura', 'Nièvre', 'Haute-Saône', 'Saône-et-Loire', 'Yonne', 'Territoire de Belfort'],
    'Bretagne': ['Côtes-d\'Armor', 'Finistère', 'Ille-et-Vilaine', 'Morbihan'],
    'Centre-Val de Loire': ['Cher', 'Eure-et-Loir', 'Indre', 'Indre-et-Loire', 'Loir-et-Cher', 'Loiret'],
    # 'Corse': ['Corse-du-Sud', 'Haute-Corse'],
    'Grand Est': ['Ardennes', 'Aube', 'Marne', 'Haute-Marne', 'Meurthe-et-Moselle', 'Meuse', 'Moselle', 'Bas-Rhin', 'Haut-Rhin', 'Vosges'],
    'Hauts-de-France': ['Aisne', 'Nord', 'Oise', 'Pas-de-Calais', 'Somme'],
    'Île-de-France': ['Paris', 'Seine-et-Marne', 'Yvelines', 'Essonne', 'Hauts-de-Seine', 'Seine-Saint-Denis', 'Val-de-Marne', 'Val-d\'Oise'],
    'Normandie': ['Calvados', 'Eure', 'Manche', 'Orne', 'Seine-Maritime'],
    'Nouvelle-Aquitaine': ['Charente', 'Charente-Maritime', 'Corrèze', 'Creuse', 'Dordogne', 'Gironde', 'Landes', 'Lot-et-Garonne', 'Pyrénées-Atlantiques', 'Deux-Sèvres', 'Vienne', 'Haute-Vienne'],
    'Occitanie': ['Ariège', 'Aude', 'Aveyron', 'Gard', 'Haute-Garonne', 'Gers', 'Hérault', 'Lot', 'Lozère', 'Hautes-Pyrénées', 'Pyrénées-Orientales', 'Tarn', 'Tarn-et-Garonne'],
    'Pays de la Loire': ['Loire-Atlantique', 'Maine-et-Loire', 'Mayenne', 'Sarthe', 'Vendée'],
    'Provence-Alpes-Côte d\'Azur': ['Alpes-de-Haute-Provence', 'Hautes-Alpes', 'Alpes-Maritimes', 'Bouches-du-Rhône', 'Var', 'Vaucluse']
}

Regions = [
    'Auvergne-Rhône-Alpes',
    'Bourgogne-Franche-Comté',
    'Bretagne',
    'Centre-Val de Loire',
    # 'Corse',
    'Grand Est',
    'Hauts-de-France',
    'Île-de-France',
    'Normandie',
    'Nouvelle-Aquitaine',
    'Occitanie',
    'Pays de la Loire',
    'Provence-Alpes-Côte d\'Azur'
]

Name2num = {
    'Ain': '01',
    'Aisne': '02',
    'Allier': '03',
    'Alpes-de-Haute-Provence': '04',
    'Hautes-Alpes': '05',
    'Alpes-Maritimes': '06',
    'Ardèche': '07',
    'Ardennes': '08',
    'Ariège': '09',
    'Aube': '10',
    'Aude': '11',
    'Aveyron': '12',
    'Bouches-du-Rhône': '13',
    'Calvados': '14',
    'Cantal': '15',
    'Charente': '16',
    'Charente-Maritime': '17',
    'Cher': '18',
    'Corrèze': '19',
    'Corse-du-Sud': '2A',
    'Haute-Corse': '2B',
    'Côte-d\'Or': '21',
    'Côtes-d\'Armor': '22',
    'Creuse': '23',
    'Dordogne': '24',
    'Doubs': '25',
    'Drôme': '26',
    'Eure': '27',
    'Eure-et-Loir': '28',
    'Finistère': '29',
    'Gard': '30',
    'Haute-Garonne': '31',
    'Gers': '32',
    'Gironde': '33',
    'Hérault': '34',
    'Ille-et-Vilaine': '35',
    'Indre': '36',
    'Indre-et-Loire': '37',
    'Isère': '38',
    'Jura': '39',
    'Landes': '40',
    'Loir-et-Cher': '41',
    'Loire': '42',
    'Haute-Loire': '43',
    'Loire-Atlantique': '44',
    'Loiret': '45',
    'Lot': '46',
    'Lot-et-Garonne': '47',
    'Lozère': '48',
    'Maine-et-Loire': '49',
    'Manche': '50',
    'Marne': '51',
    'Haute-Marne': '52',
    'Mayenne': '53',
    'Meurthe-et-Moselle': '54',
    'Meuse': '55',
    'Morbihan': '56',
    'Moselle': '57',
    'Nièvre': '58',
    'Nord': '59',
    'Oise': '60',
    'Orne': '61',
    'Pas-de-Calais': '62',
    'Puy-de-Dôme': '63',
    'Pyrénées-Atlantiques': '64',
    'Hautes-Pyrénées': '65',
    'Pyrénées-Orientales': '66',
    'Bas-Rhin': '67',
    'Haut-Rhin': '68',
    'Rhône': '69',
    'Haute-Saône': '70',
    'Saône-et-Loire': '71',
    'Sarthe': '72',
    'Savoie': '73',
    'Haute-Savoie': '74',
    'Paris': '75',
    'Seine-Maritime': '76',
    'Seine-et-Marne': '77',
    'Yvelines': '78',
    'Deux-Sèvres': '79',
    'Somme': '80',
    'Tarn': '81',
    'Tarn-et-Garonne': '82',
    'Var': '83',
    'Vaucluse': '84',
    'Vendée': '85',
    'Vienne': '86',
    'Haute-Vienne': '87',
    'Vosges': '88',
    'Yonne': '89',
    'Territoire de Belfort': '90',
    'Essonne': '91',
    'Hauts-de-Seine': '92',
    'Seine-Saint-Denis': '93',
    'Val-de-Marne': '94',
    'Val-d\'Oise': '95'
}


In [ ]:
selected_regions = [
    'Marne'
    # 'Grand Est', 
    # 'Hauts-de-France',
    # 'Bourgogne-Franche-Comté',
    # 'Auvergne-Rhône-Alpes',
    # 'Provence-Alpes-Côte d\'Azur'

]

In [ ]:
try:
    selected_departements = []
    for reg in selected_regions:
        selected_departements += Administratives[reg]
except:
    print('Not a Région')
    selected_departements= selected_regions
selected_departements

In [ ]:
def get_graph(place):
    return ox.graph_from_place(
        place,
        network_type='drive',
        simplify=True,
        clean_periphery=True,
        retain_all=False,           # trunk|
        custom_filter='["highway"~"^(primary|primary_link|secondary|secondary_link|motorway|motorway_link|trunk|trunk_link)$"]',
        # custom_filter='["ref"~"^(D [0-9]|A [0-9]|N [0-9])$"]',
        # custom_filter='["ref"~"^(\s*[A]\s*[0-9]*\s*)$"]',
        # custom_filter='["ref"~"^(\s*[D]\s*[0-9]*\s*)$"]',
        # custom_filter='["ref"~"^(\s*[ADN]\s*[0-9]*\s*[a-zA-Z]*\s*[0-9]*\s*)$"]',
        # custom_filter='["highway"!~"^(tertiary|tertiary_link)$"]',


        # custom_filter='["ref"~"^D [0-9]+$"|"ref"~"A [0-9]+$"|"ref"~"N [0-9]+$"|"highway"~"motorway"]',
    )


try:
    print(selected_regions)
    ox.settings.cache_only_mode=True 
    G = get_graph(selected_regions)
except:
    print('All requests done, building graph...')
    ox.settings.cache_only_mode=False 
    G = get_graph(selected_regions)


G.number_of_nodes(), G.number_of_edges()   

In [ ]:
# r = []
# errors = 0
# for n in G.edges():
#     try:
#         r.append(str(G.get_edge_data(*n)[0]['ref']))
#     except:
#         errors += 1
# set(r),errors        

In [ ]:
# r = []
# errors = 0
# for n in G.edges():
#     try:
#         r.append(str(G.get_edge_data(*n)[0]['highway']))
#     except:
#         errors += 1
# set(r),errors

In [ ]:
# query = ox.downloader._create_overpass_query('Marne France',
#     {
#     'amenity': 'charging_station', 
#     # 'highway': 'trunk|primary|secondary|tertiary|motorway|motorway_link'
#     }
# )
# charge = ox.geometries.geometries_from_place(
#     'Marne, France', 
#     tags={'amenity': 'charging_station'}, 
#     which_result=None, 
#     buffer_dist=None)
# len(charge.columns)    
# charge.iloc[:,:33].T
# charge.iloc[:,34:].T

In [ ]:
G = ox.simplification._consolidate_intersections_rebuild_graph(
    G, tolerance=1e-4, reconnect_edges=True
    )
G.number_of_nodes(), G.number_of_edges()    

In [ ]:
# impute edge (driving) speeds and calculate edge traversal times
G = ox.add_edge_speeds(G)
G = ox.distance.add_edge_lengths(G)
# G = ox.add_edge_travel_times(G)
nx.set_node_attributes(G, values=(0.,0.,1.,0.), name='color')

In [ ]:
# Define the file path for the compressed graph file
filepath = f"./datas/{selected_regions}_drive.graphml"

# Save the graph to disk with compression
ox.save_graphml(G, filepath=filepath)

In [ ]:

nc = [d['color'] for idx, d in G.nodes(data=True)]
ox.plot_graph(G, node_color=nc)

In [ ]:
G.number_of_nodes(), G.number_of_edges()

In [ ]:
import folium
from folium.plugins import MarkerCluster

m = ox.folium.plot_graph_folium(G, tiles='OpenStreetMap')
group = MarkerCluster(name="Nodes")
for (n, x), (_,y) in zip(nx.get_node_attributes(G,'x').items(), nx.get_node_attributes(G,'y').items()):
    group.add_child(
        folium.Marker(
            location=[y, x],
            popup=n, icon=folium.Icon(color='red')
            )
        )
group.add_to(m)
folium.LayerControl().add_to(m) 

display(m)
del m

In [ ]:
import requests

session = requests.Session()
headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}
session.headers.update(headers)

def get_(url):
    response = session.get(url.strip())
    return response.json()
# GET COORDINATES FROM CITY NAME

def get_place_polygon(county,city, country):
    url  =  'https://nominatim.openstreetmap.org/search?'
    # url += f'q={adress}%2C+'
    url += f'city={city}&'
    url += f'country={country}&'
    url += f'county={county}&'


    url +=  'format=geojson&polygon_geojson=1'
    return get_(url)

In [ ]:
# # import osmnx as ox
# # import pandas as pd
# # import geopandas as gpd


# # # Create an empty GeoDataFrame to store the département polygons
# # df = gpd.GeoDataFrame()

# # # Loop through each region and retrieve the corresponding département polygons
# # for dep in departements:
# #     query = f'({dep}, France'
# #     dep_gdf = ox.geocode_to_gdf(query)
# #     # region_gdf['region'] = region_name
# #     df = pd.concat([df, dep_gdf], axis=0, ignore_index=True)

# # df, set(df.loc[:,'type'].values)
# set(df.loc[:,'type'].values)

In [ ]:
# area = get_place_polygon(
#         '51', 
#         '', 
#         'France'
#         ) 
# area = area['features'][0]['geometry']['coordinates'][0]
# area

In [ ]:
import polyline
from shapely.geometry import Point,Polygon

class ChargingStation:
    def __init__(self, feature_collection, idx=0):
        AddressInfo = feature_collection['AddressInfo']
        Connections = feature_collection['Connections']

        self.AddressInfo = AddressInfo # dict
        self.Connections = Connections # list of dicts

        self.id = feature_collection['ID']
        self.uuid = feature_collection['UUID']
        self.provider = feature_collection['DataProvider']['Title']
        self.address = AddressInfo['AddressLine1']
        self.point = Point(AddressInfo['Longitude'], AddressInfo['Latitude'])
        self.nbr_connections = len(Connections)
        try:
            self.level = Connections[idx]['Level']['IsFastChargeCapable'] 
        except:
            self.level = None
        try:
            self.power = Connections[idx]['PowerKW'] 
        except:
            self.power = 0
        try:
            self.type = Connections[idx]['CurrentType']['Title']
        except:
            self.type = None



def request_openchargemap(querystring, output=-1):
    url = "https://api.openchargemap.io/v3/poi"
    headers = {"Accept": "application/json"}
    
    with requests.Session() as s:
        response = s.get(url, headers=headers, params=querystring)
    # print(response, end='\r')

    if response.status_code == 200:
        # display(response.content)
        try:
            datas = response.json()
        except:
            datas = {}

        if datas != {}:    
            # list of available infos
            if output > -1:
                for n, data in enumerate(datas):
                    for key in data.keys():
                        if n == output : print(f'[{n}]-key {key} : {data[key]}')
            
            # list of charging stations
            charging_stations = []
            count = 0
            for n, data in enumerate(datas):
                plugs = len(data['Connections'])
                if output > -1: print(f'{n + 1} with {plugs} plugs / {len(datas)} stations')
                if data['DataQualityLevel'] >0:
                    # TODO handle data quality
                    for n in range(plugs):
                        count += 1
                        charging_stations.append(
                            ChargingStation( data, idx=n )
                        )
            # try:
            #     print(f'Number of total plugs: {count}')   
            # except:
            #     raise Exception('No charging stations returned')
            if len(charging_stations) > 0:
                return charging_stations 
            else:
                return []   #raise Exception('No valid charging stations found')
        else:
            return []     #raise Exception(response.text)
    return[]


import time

def get_charging_stations(area, maxresults=5000):

    querystring = {
        "maxresults": f"{maxresults}",
        "countrycode": "80",
        # "latitude": string
        # "longitude": string
        # "distance": f"{distance}",
        # "distanceunit": "km",
        # "opendata": "true",
        # "polyline": google polyline encoded: string
        "key": "42fc0eed-7a1d-49f6-b7a2-724b136adb3b"
        }   

    try:
        # Nominatim coordinates Multi polygons    
        stations = []
        for sub_area in area: 
            for sub in sub_area:
                simplified = Polygon(sub).simplify(0.01, preserve_topology=True)
                simplified_enc = polyline.encode(simplified.exterior.coords, precision=5, geojson=True) 
                querystring['polygon'] = f"{simplified_enc}" 
                stations.extend(request_openchargemap(querystring))
            
    except:        
        # Nominatim coordinates polygon
        # print(area)
        simplified = Polygon(area[0]).simplify(0.01, preserve_topology=True)
        simplified_enc = polyline.encode(simplified.exterior.coords, precision=5, geojson=True) 
        querystring['polygon'] = f"{simplified_enc}" 
        stations = request_openchargemap(querystring)

    ## osmnx shapely polygons
    # try:
    #     areas = [g for g in area.geoms]
    # except:
    #     areas = [area]    
    # stations = []
    # for n, area in enumerate(tqdm(areas, leave=True)):
    #     # print(f'     {n + 1}/{len(areas)}', end='\r')
    #     simplified = area.exterior.simplify(0.01, preserve_topology=True)
    #     simplified_enc = polyline.encode(simplified.coords, precision=5, geojson=True) 
    #     querystring['polygon'] = f"{simplified_enc}" 
    #     stations.extend(request_openchargemap(querystring))
    
    # print('ok')  
    print(len(stations))  
    return stations


In [ ]:
areas = [get_place_polygon(
                    county=Name2num[dept], 
                    city='', 
                    country='France'
            )['features'][0]['geometry']['coordinates'] for dept in selected_departements 
]
len(areas)

In [ ]:
stations = [get_charging_stations(area) for area in tqdm(areas, leave=False)]
len(stations)

In [ ]:
def flatten_list(l):
    return [item for sublist in l for item in sublist]
len(flatten_list(stations))

In [ ]:
def unique_station(stations):
    st = flatten_list(stations)
    remain = []
    for n in tqdm(range(len(st))):
        test = st.pop(0)
        keep = True
        for n in st:
            if test.__dict__  == n.__dict__:
                keep = False
                break
        if keep:    
            remain.append(test)
    print(f'Unique Charge Stations (with multi points): {len(remain)}')         
    return remain
        
unique_stations = unique_station(stations)      

In [ ]:
def remove_duplicate_stations(stations):
    keep = dict()
    for n, station in enumerate(tqdm(stations)):
        keep[station.id] = (n)

    return [stations[keep[key]] for key in keep.keys()]
len(remove_duplicate_stations(flatten_list(stations)))

In [ ]:
# G1 = ox.graph_from_point( (stations[0].point.y, stations[0].point.x) ,
#     dist = 1000,
#     network_type='drive',
#     simplify=True,
#     retain_all=True,
#     )

# ox.plot_graph(G1)

In [ ]:
# (closest_node, next_node, key), dist = ox.distance.nearest_edges(G1, X=stations[0].point.x, Y=stations[0].point.y, return_dist=True)
# (closest_node, next_node, key), dist
# G1.nodes[closest_node],G1.edges[closest_node, next_node, key]['geometry'].coords[0],
# G1.nodes[next_node],G1.edges[closest_node, next_node, key]['geometry'].coords[-1],

In [ ]:
from shapely.geometry import LineString
import pyproj
dist_meter = lambda x : pyproj.Geod(ellps='WGS84').geometry_length(x)

def split_line(point, linestring):
    best_distance = 9999
    best_idx = -1
    for idx, line_point in enumerate(linestring.coords):
        dist = point.distance( Point(line_point))
        if dist < best_distance:
            best_distance = dist
            best_idx = idx
        else:
            break
    start_line =  linestring.coords[:idx]
    start_line.append(point.coords[0])
    end_line = [point.coords[0]]
    end_line.extend(linestring.coords[idx:])
    return LineString(start_line), LineString(end_line)

def graph_add_stations(graph, stations):
    errors = 0
    for n, station in enumerate(tqdm(stations)):
        # print(f'{n+1}/{len(stations)}', end='\r')
        point = station.point
        # print(station.point, end='  ')
        # Get from/to id of closest edge
        (closest_node, next_node, key), dist = ox.distance.nearest_edges(graph, X=station.point.x, Y=station.point.y, return_dist=True)

        # print(closest_node, next_node, key)
        if str(closest_node) != closest_node or dist != 0:
            s = f"station_{station.AddressInfo['ID']}"      #id of new node

            edge_attrs = graph.edges[closest_node, next_node, key] # dict of edge attributes
            # print(edge_attrs)
            try:
                edge_linestring = edge_attrs['geometry']
            except:
                continue
            closest_point = edge_linestring.interpolate(edge_linestring.project(station.point))

            closest_node_line, next_node_line = split_line(closest_point, edge_linestring) 

            graph.add_node(s, 
                        y= closest_point.y, 
                        x= closest_point.x, 
                        real= Point(station.point.x,station.point.y), 
                        street_count = 2,
                        color = (1.,0.,0.,1.),
                        infos = station.__dict__)

            
            graph.add_edge(closest_node, s,key,
                        osmid = edge_attrs['osmid'],
                        # ref = edge_attrs['ref'],
                        # name = edge_attrs['name'],
                        highway = edge_attrs['highway'],
                        oneway =  edge_attrs['oneway'],
                        reversed =  edge_attrs['reversed'],
                        length =  dist_meter(closest_node_line),
                        geometry = closest_node_line,
                        speed_kph = edge_attrs['speed_kph'],
                        # travel_time = edge_attrs['travel_time']
            )
            graph.add_edge(s, closest_node, key,
                        osmid = edge_attrs['osmid'],
                        # ref = edge_attrs['ref'],
                        # name = edge_attrs['name'],
                        highway = edge_attrs['highway'],
                        oneway =  edge_attrs['oneway'],
                        reversed =  edge_attrs['reversed'],
                        length =  dist_meter(closest_node_line),
                        geometry = closest_node_line,
                        speed_kph = edge_attrs['speed_kph'],
                        # travel_time = edge_attrs['travel_time']
            )

            graph.add_edge(s, next_node, key,
                        osmid = edge_attrs['osmid'],
                        # ref = edge_attrs['ref'],
                        # name = edge_attrs['name'],
                        highway = edge_attrs['highway'],
                        oneway =  edge_attrs['oneway'],
                        reversed =  edge_attrs['reversed'],
                        length =  dist_meter(next_node_line),
                        geometry = next_node_line,
                        speed_kph = edge_attrs['speed_kph'],
                        # travel_time = edge_attrs['travel_time']
            )
            graph.add_edge(next_node, s, key,
                        osmid = edge_attrs['osmid'],
                        # ref = edge_attrs['ref'],
                        # name = edge_attrs['name'],
                        highway = edge_attrs['highway'],
                        oneway =  edge_attrs['oneway'],
                        reversed =  edge_attrs['reversed'],
                        length =  dist_meter(next_node_line),
                        geometry = next_node_line,
                        speed_kph = edge_attrs['speed_kph'],
                        # travel_time = edge_attrs['travel_time']
            )
            try:
                graph.remove_edge(closest_node, next_node, key)
            except:
                errors +=1
            try:
                graph.remove_edge(next_node, closest_node, key)
            except:
                errors +=1

        else:
            print('duplicate station node: ',closest_node,f"[station_{station.AddressInfo['ID']}]")
    return graph, errors

G1, errors = graph_add_stations(G.copy(), unique_stations)  

G1.number_of_nodes(), G1.number_of_edges(), errors

In [ ]:
G1.number_of_nodes() - G.number_of_nodes(), G1.number_of_edges() - G.number_of_edges()

In [ ]:
# Define the file path for the compressed graph file
filepath = f"./datas/{selected_regions}_drive_charge.graphml"

# Save the graph to disk with compression
ox.save_graphml(G1, filepath=filepath)

In [ ]:
edge_colors = ox.plot.get_edge_colors_by_attr(
    G1, 
    'speed_kph', 
    num_bins=None, 
    cmap='viridis', 
    # start=0, 
    # stop=10, 
    na_color='none', 
    equal_size=False
)
G1.number_of_edges() - len(edge_colors)

In [ ]:
res = []
errors = []
for idx in G1.nodes():
    try:
        res.append(G1.nodes[idx]['color'])
    except:
        errors.append(idx)
len(res),len(errors),  G1.number_of_nodes()-len(res)-len(errors)   

In [ ]:
# node_colors = ox.plot.get_node_colors_by_attr(
#     G1,
#     'color',
#     # num_bins=3,
#     cmap='CMRmap',
#     # start=0,
#     # stop=10,
#     # na_color=None,
#     # equal_size=False
# )
# G1.number_of_nodes() - len(node_colors)

In [ ]:
# for idx, d in G.nodes(data=True):
#     print(idx, d)
#     break

In [ ]:
# plot nodes in red if charge point else blue
nc = [d['color'] for idx, d in G1.nodes(data=True)]
len(nc), set(nc)

In [ ]:
ox.plot_graph(G1,
    node_color=nc, 
    node_size=10,
    node_alpha=1.0,
    edge_color=edge_colors,
    edge_linewidth=1,
    edge_alpha=0.5
    )

In [ ]:
import folium

m = ox.folium.plot_graph_folium(G1, tiles='OpenStreetMap')
# group = folium.FeatureGroup(name="Stations_real")
# for key, value in nx.get_node_attributes(G1,'real').items():
#     group.add_child(
#         folium.Marker(
#             location=[value.y, value.x],
#             popup=key, icon=folium.Icon(color='red')
#             )
#         )
# group.add_to(m)
# folium.LayerControl().add_to(m) 

Sgroup = MarkerCluster(name="Station Position")
Ngroup = MarkerCluster(name="Station Nodes")

for  (n,p) in nx.get_node_attributes(G1,'real').items():
    Sgroup.add_child(
        folium.Marker(
            location=[p.y, p.x],
            popup=n, icon=folium.Icon(color='red')
            )
        )

    x = nx.get_node_attributes(G1,'x')[n]
    y = nx.get_node_attributes(G1,'y')[n]
    Ngroup.add_child(
        folium.Marker(
            location=[y, x],
            popup=n, icon=folium.Icon(color='blue')
            )
        )
Sgroup.add_to(m)
Ngroup.add_to(m)
folium.LayerControl().add_to(m) 

display(m)
del m

In [ ]:
nx.get_node_attributes(G1,'x').keys()

-----------------------------

In [ ]:
*

In [ ]:
def split_dict(dct):
    dct_stations={}
    dct_nodes = {}
    for key, value in dct.items():
        if str(key) == key:
            dct_stations[key] =  value
        else:
            dct_nodes[key] = value    
    display( len(dct_stations), len(dct_nodes))
    return dct_stations, dct_nodes

In [ ]:
closeness = nx.closeness_centrality(
                G, 
                u=None, 
                distance='distance', 
                wf_improved=True
                )
clsns_stations, clsns_nodes = split_dict(closeness)

In [ ]:
betweeness = nx.betweenness_centrality(
    G1, 
    k=50, 
    normalized=True, 
    weight='distance', 
    endpoints=False, 
    seed=42)

btwns_stations, btwns_nodes = split_dict(betweeness)

-------------------------------------------------

In [ ]:
closeness

In [ ]:
clsns_stations

In [ ]:
btwns_stations

In [ ]:
nx.get_node_attributes(G1,'real')

In [ ]:
nx.get_node_attributes(G1,'infos')['station_19525']

In [ ]:
nx.get_node_attributes(G1,'infos')['station_199666']

In [ ]:
betweeness

--------------------------------------------